<a href="https://colab.research.google.com/github/YHWOO13/JBNU-2017-Fall/blob/master/Thesis/%5BMASK%5D%5BBERT_Approach%5D_Reliability_of_relation_in_KG_v2_4_cuML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore the relation reliability with hopping (Enhanced contextualized)

In [1]:
%%capture
!pip install pykeen
!pip install permetrics
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [2]:
import cuml
cuml.__version__

'25.04.00'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import requests
import networkx as nx
import numpy as np
import pandas as pd
import pykeen
import torch
import json
import random
import torch.nn.functional as F
import matplotlib as plt
import time
import cupy as cp

# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score
from sklearn.metrics import normalized_mutual_info_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

from collections import Counter
from collections import defaultdict
from permetrics import ClusteringMetric

from cuml.cluster import KMeans
from cuml.metrics.cluster import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt


from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
# from pykeen.pipeline import pipeline
# from pykeen.triples import TriplesFactory

In [ ]:
import nltk
from nltk.corpus import wordnet as wn

nltk.download('wordnet')

In [6]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Get the mapping of entities to their IDs
entity_to_id = umls.entity_to_id
relaton_to_id = umls.relation_to_id
# List all entities
umls_entities = list(entity_to_id.keys())
print(f"Number of entities: {len(umls_entities)}")
print("Entities:", umls_entities)

umls_relations = list(relaton_to_id.keys())
print(f"Number of relations: {len(umls_relations)}")
print("Relations:", umls_relations)


Number of entities: 135
Entities: ['acquired_abnormality', 'activity', 'age_group', 'alga', 'amino_acid_peptide_or_protein', 'amino_acid_sequence', 'amphibian', 'anatomical_abnormality', 'anatomical_structure', 'animal', 'antibiotic', 'archaeon', 'bacterium', 'behavior', 'biologic_function', 'biologically_active_substance', 'biomedical_occupation_or_discipline', 'biomedical_or_dental_material', 'bird', 'body_location_or_region', 'body_part_organ_or_organ_component', 'body_space_or_junction', 'body_substance', 'body_system', 'carbohydrate', 'carbohydrate_sequence', 'cell', 'cell_component', 'cell_function', 'cell_or_molecular_dysfunction', 'chemical', 'chemical_viewed_functionally', 'chemical_viewed_structurally', 'classification', 'clinical_attribute', 'clinical_drug', 'conceptual_entity', 'congenital_abnormality', 'daily_or_recreational_activity', 'diagnostic_procedure', 'disease_or_syndrome', 'drug_delivery_device', 'educational_activity', 'eicosanoid', 'element_ion_or_isotope', 'emb

# From saved existing/non-existing triples

In [7]:
import json
with open('/content/drive/MyDrive/Colab Notebooks/Thesis/existing_list.json', 'r') as file:
    existing_lst = json.load(file)

with open('/content/drive/MyDrive/Colab Notebooks/Thesis/non_existing_list.json', 'r') as file:
    non_existing_lst = json.load(file)

# Remove labels (existing/non-existing) and keep (subject, relation, object)
non_existing_triples = [item[:-1] for item in non_existing_lst]
existing_triples = [item[:-1] for item in existing_lst]

In [ ]:
from transformers import pipeline
import torch

# Step 1: Filter triples
lst = []
for triple in existing_triples:
    subj, rel, obj = triple
    if subj == 'receptor' and obj == 'neoplastic_process':
        lst.append(triple)

print(f"✅ Found {len(lst)} matching triples:")
for t in lst:
    print(t)

# Step 2: Load BERT fill-mask model
fill_mask = pipeline("fill-mask", model="bert-base-uncased", device=0 if torch.cuda.is_available() else -1)

# Step 3: Get BERT predictions for each (subj, obj) pair
print("\n===== BERT Predictions =====")
for subj, rel, obj in lst:
    mask_prompt = f"{subj} {fill_mask.tokenizer.mask_token} {obj}"
    print(f"\n🔍 Masked prompt: {mask_prompt}")

    try:
        predictions = fill_mask(mask_prompt, top_k=5)  # you can change top_k as needed
        print("  Top predictions:")
        for p in predictions:
            print(f"    {p['token_str']} (score: {p['score']:.4f})")
    except Exception as e:
        print(f"  ❌ Error: {e}")


✅ Found 3 matching triples:
['receptor', 'complicates', 'neoplastic_process']
['receptor', 'causes', 'neoplastic_process']
['receptor', 'affects', 'neoplastic_process']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0



===== BERT Predictions =====

🔍 Masked prompt: receptor [MASK] neoplastic_process
  Top predictions:
    _ (score: 0.6575)
    - (score: 0.0538)
    / (score: 0.0412)
    and (score: 0.0203)
    : (score: 0.0140)

🔍 Masked prompt: receptor [MASK] neoplastic_process
  Top predictions:
    _ (score: 0.6575)
    - (score: 0.0538)
    / (score: 0.0412)
    and (score: 0.0203)
    : (score: 0.0140)

🔍 Masked prompt: receptor [MASK] neoplastic_process
  Top predictions:
    _ (score: 0.6575)
    - (score: 0.0538)
    / (score: 0.0412)
    and (score: 0.0203)
    : (score: 0.0140)


In [9]:
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel, pipeline

import torch
from tqdm import tqdm

In [10]:
# Load model and tokenizer (swap to Roberta* if needed)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)
model.eval()  # always good practice

# tokenizer2 = RobertaTokenizer.from_pretrained('roberta-base')
# model2 = RobertaModel.from_pretrained('roberta-base').to(device)
# model2.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
unique_relations = list(set([r for _, r, _ in existing_triples]))

existing_vocab = tokenizer.get_vocab()

already_in_vocab = [r for r in unique_relations if r in existing_vocab]
newly_added = [r for r in unique_relations if r not in existing_vocab]

print("Already in vocab:", len(already_in_vocab), already_in_vocab)
print("Will be added:", len(newly_added), newly_added)



Already in vocab: 15 ['manages', 'treats', 'produces', 'measures', 'performs', 'affects', 'uses', 'isa', 'exhibits', 'contains', 'practices', 'surrounds', 'prevents', 'causes', 'indicates']
Will be added: 31 ['complicates', 'developmental_form_of', 'property_of', 'occurs_in', 'location_of', 'adjacent_to', 'issue_in', 'disrupts', 'conceptually_related_to', 'consists_of', 'diagnoses', 'method_of', 'evaluation_of', 'degree_of', 'manifestation_of', 'analyzes', 'assesses_effect_of', 'part_of', 'ingredient_of', 'interacts_with', 'conceptual_part_of', 'precedes', 'carries_out', 'associated_with', 'connected_to', 'derivative_of', 'result_of', 'process_of', 'co-occurs_with', 'interconnects', 'measurement_of']


In [ ]:
# Example sentence
sentence = "diagnostic procedure method_of occupation or discipline."
masked = "diagnostic procedure [MASK]  occupation or discipline."
template = "diagnostic procedure is a method of  occupation or discipline."
# Tokenize manually

print("BERT")
tokens = tokenizer.tokenize(sentence)
masked_tokens = tokenizer.tokenize(masked)
template_tokens = tokenizer.tokenize(template)
print(tokens)
print(masked_tokens)
print(template_tokens)

# print('roberta')
# tokens2 = tokenizer2.tokenize(sentence)
# masked_tokens2 = tokenizer2.tokenize(masked)
# template_tokens2 = tokenizer2.tokenize(template)
# print(tokens2)
# print(masked_tokens2)
# print(template_tokens2)

BERT
['diagnostic', 'procedure', 'method', '_', 'of', 'occupation', 'or', 'discipline', '.']
['diagnostic', 'procedure', '[MASK]', 'occupation', 'or', 'discipline', '.']
['diagnostic', 'procedure', 'is', 'a', 'method', 'of', 'occupation', 'or', 'discipline', '.']


# Add tokens

In [ ]:
# Step 1: Extract unique relation tokens
unique_relations = list(set([r for _, r, _ in existing_triples]))
print(len(unique_relations))
# Step 2: Add the relation tokens to the tokenizer
model.tokenizer.add_tokens(unique_relations)

# Step 3: Resize model embeddings to accommodate new tokens
model._first_module().auto_model.resize_token_embeddings(len(model.tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


46


Embedding(30553, 384, padding_idx=0)

In [ ]:
relation_templates = {
    "adjacent_to":            "X is adjacent to Y",
    "affects":                "X affects Y",
    "analyzes":               "X analyzes Y",
    "assesses_effect_of":     "X assesses the effect of Y",
    "associated_with":        "X is associated with Y",
    "carries_out":            "X carries out Y",
    "causes":                 "X causes Y",
    "co-occurs_with":         "X co-occurs with Y",
    "complicates":            "X complicates Y",
    "conceptual_part_of":     "X is conceptually part of Y",
    "conceptually_related_to": "X is conceptually related to Y",
    "connected_to":           "X is connected to Y",
    "consists_of":            "X consists of Y",
    "contains":               "X contains Y",
    "degree_of":              "X has a degree of Y",  # (may need further refinement)
    "derivative_of":          "X is a derivative of Y",
    "developmental_form_of":  "X is a developmental form of Y",
    "diagnoses":              "X diagnoses Y",
    "disrupts":               "X disrupts Y",
    "evaluation_of":          "X is an evaluation of Y",
    "exhibits":               "X exhibits Y",
    "indicates":              "X indicates Y",
    "ingredient_of":          "X is an ingredient of Y",
    "interacts_with":         "X interacts with Y",
    "interconnects":          "X interconnects Y",
    "isa":                    "X is a Y",
    "issue_in":               "X is an issue in Y",
    "location_of":            "X is the location of Y",
    "manages":                "X manages Y",
    "manifestation_of":       "X is a manifestation of Y",
    "measurement_of":         "X is a measurement of Y",
    "measures":               "X measures Y",
    "method_of":              "X is a method of Y",
    "occurs_in":              "X occurs in Y",
    "part_of":                "X is a part of Y",
    "performs":               "X performs Y",
    "practices":              "X practices Y",
    "precedes":               "X precedes Y",
    "prevents":               "X prevents Y",
    "process_of":             "X is a process of Y",
    "produces":               "X produces Y",
    "property_of":            "X is a property of Y",
    "result_of":              "X is a result of Y",
    "surrounds":              "X surrounds Y",
    "treats":                 "X treats Y",
    "uses":                   "X uses Y"
}

# 040

# Think of "method_of" like a multi-word expression. We want the combined meaning, not just one word.

Averaging is a quick way to approximate:

"What is the overall semantic signal of this phrase, in this specific sentence?"

And BERT already encodes the context for both words — so averaging them tends to give a very reasonable and usable representation.

In [16]:
relation_labels = [r for _, r, _ in existing_triples]

Relations = set(list(relation_labels))

relations = [r for r in Relations if r != 'isa']  # Exclude 'isa'

print(len(relations))


# Step 1: Keep only relations with <2 underscores
core_rel_relations = [r for r in relations if r.count('_') < 2]
two_underline_relations = [r for r in relations if r.count('_') >= 2]

# Step 2: Keep only triples with those relations
full_triples = [t for t in existing_triples if t[1] in relations]
core_rel_triples = [t for t in existing_triples if t[1] in core_rel_relations]
revised_triples = [t for t in existing_triples if t[1] in two_underline_relations]

print('core_rel_relations', len(core_rel_relations), core_rel_relations)
print('two_underline_relations', two_underline_relations)
print('core_rel_triples', len(core_rel_triples))
print(len(revised_triples))
print(len(full_triples))

45
core_rel_relations 41 ['interacts_with', 'performs', 'consists_of', 'degree_of', 'carries_out', 'surrounds', 'measurement_of', 'analyzes', 'practices', 'process_of', 'evaluation_of', 'adjacent_to', 'interconnects', 'derivative_of', 'manifestation_of', 'issue_in', 'result_of', 'complicates', 'occurs_in', 'manages', 'ingredient_of', 'co-occurs_with', 'treats', 'causes', 'indicates', 'method_of', 'property_of', 'produces', 'associated_with', 'affects', 'connected_to', 'precedes', 'disrupts', 'contains', 'part_of', 'exhibits', 'prevents', 'measures', 'uses', 'diagnoses', 'location_of']
two_underline_relations ['conceptual_part_of', 'assesses_effect_of', 'conceptually_related_to', 'developmental_form_of']
core_rel_triples 5940
89
6029


In [ ]:
'''
masked = f"{s_clean} [MASK] {o_clean}."

rel: method_of
tokens_mask ['[CLS]', 'diagnostic', 'procedure', '[MASK]', 'occupation', 'or', 'discipline', '.', '[SEP]', '[PAD]']
mask_indices [3]

rel: affects
tokens_mask ['[CLS]', 'molecular', 'biology', 'research', 'technique', '[MASK]', 'mental', 'process', '.', '[SEP]']
mask_indices [5]

rel: result_of
tokens_mask ['[CLS]', 'mental', 'process', '[MASK]', 'behavior', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]']
mask_indices [3]

rel: affects
tokens_mask ['[CLS]', 'research', 'activity', '[MASK]', 'mental', 'process', '.', '[SEP]', '[PAD]', '[PAD]']
mask_indices [3]

'''

# Full Relation & Masking

In [ ]:
def extract_relation_embeddings(
    triples, tokenizer, model, batch_size=128, device="cuda"
):
    """
    Extracts full and masked embeddings for relation triples.

    Args:
        triples (list of tuples): List of (subject, relation, object) triples.
        tokenizer: HuggingFace tokenizer.
        model: HuggingFace transformer model.
        batch_size (int): Batch size for processing.
        device (str): Device to run model on ("cuda" or "cpu").

    Returns:
        X_full (list): Embeddings for unmasked (full) relations.
        X_masked (list): Embeddings for masked relations.
        y (list): List of relation labels.
    """
    def clean_text(text):
        return text.replace("_", " ")

    X_full, X_masked, y = [], [], []

    for i in tqdm(range(0, len(triples), batch_size), desc="Extracting embeddings"):
        batch = triples[i:i + batch_size]

        sent_list = []
        mask_list = []
        rel_list = []

        for s, rel, o in batch:
            s_clean = clean_text(s)
            o_clean = clean_text(o)
            full_rel = rel.replace("_", " ")
            sent = f"{s_clean} {full_rel} {o_clean}."
            masked = f"{s_clean} [MASK] {o_clean}."

            sent_list.append(sent)
            mask_list.append(masked)
            rel_list.append(rel)

        inputs_sent = tokenizer(sent_list, padding=True, truncation=True, return_tensors="pt").to(device)
        inputs_mask = tokenizer(mask_list, padding=True, truncation=True, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs_sent = model(**inputs_sent)
            outputs_mask = model(**inputs_mask)

        for idx, rel in enumerate(rel_list):
            rel_tokens = tokenizer.tokenize(rel.replace("_", " "))
            rel_token_ids = tokenizer.convert_tokens_to_ids(rel_tokens)

            input_ids = inputs_sent["input_ids"][idx]
            tokens = tokenizer.convert_ids_to_tokens(input_ids.cpu().tolist())

            # Find relation token span in full sentence
            match_indices = []
            for j in range(len(tokens) - len(rel_token_ids) + 1):
                if input_ids[j+1:j+1+len(rel_token_ids)].tolist() == rel_token_ids:
                    match_indices = list(range(j+1, j+1+len(rel_token_ids)))
                    break

            if match_indices:
                emb_full = outputs_sent.last_hidden_state[idx, match_indices, :].mean(dim=0).detach().cpu().numpy()
                X_full.append(emb_full)

            # Extract masked embedding
            input_ids_mask = inputs_mask["input_ids"][idx]
            tokens_mask = tokenizer.convert_ids_to_tokens(input_ids_mask.cpu().tolist())
            mask_indices = [k for k, tok in enumerate(tokens_mask) if tok == "[MASK]"]

            if mask_indices:
                emb_masked = outputs_mask.last_hidden_state[idx, mask_indices, :].mean(dim=0).detach().cpu().numpy()
                X_masked.append(emb_masked)

            y.append(rel)

    return X_full, X_masked, y

In [ ]:
print(len(y))

6029


# Core relation

In [ ]:
def extract_core_relation_embeddings(
    triples, tokenizer, model, batch_size=128, device="cuda"
):
    """
    Extracts embeddings for core relations only (e.g., first token in a relation string).

    Args:
        triples (list of tuples): List of (subject, relation, object) triples.
        tokenizer: HuggingFace tokenizer.
        model: HuggingFace transformer model.
        batch_size (int): Batch size for processing.
        device (str): Device to run model on ("cuda" or "cpu").

    Returns:
        X_core (list): Core relation embeddings.
        y_core (list): Corresponding relation labels.
    """
    def clean_text(text):
        return text.replace("_", " ")

    X_core = []
    y_core = []

    print('core_rel_triples:', len(triples))

    for i in tqdm(range(0, len(triples), batch_size), desc="Processing core_rel_triples in batches"):
        batch = triples[i:i + batch_size]

        sent_list = []
        rel_list = []

        for s, rel, o in batch:
            s_clean = clean_text(s)
            o_clean = clean_text(o)
            core_rel = rel.split("_")[0]
            sent = f"{s_clean} {core_rel} {o_clean}."

            sent_list.append(sent)
            rel_list.append(core_rel)

        # Tokenize the batch
        inputs = tokenizer(sent_list, padding=True, truncation=True, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        for idx, core_rel in enumerate(rel_list):
            rel_token_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(core_rel))
            input_ids = inputs["input_ids"][idx]
            tokens = tokenizer.convert_ids_to_tokens(input_ids.cpu().tolist())

            match_indices = []
            for j in range(len(tokens) - len(rel_token_ids) + 1):
                if input_ids[j+1:j+1+len(rel_token_ids)].tolist() == rel_token_ids:
                    match_indices = list(range(j+1, j+1+len(rel_token_ids)))
                    break

            if match_indices:
                embedding_core = outputs.last_hidden_state[idx, match_indices, :].mean(dim=0).detach().cpu().numpy()
                X_core.append(embedding_core)
                y_core.append(rel_list[idx])

    return X_core, y_core

# Calculate NMI, Purity score for Full relation / Core relaiton / Masking

In [ ]:
# Step 4: Clustering Evaluation Function
def evaluate_embeddings(embeddings, labels, label_ids, title=""):
    embeddings_gpu = cp.asarray(embeddings)
    num_clusters = len(set(label_ids))

    kmeans = KMeans(n_clusters=num_clusters, random_state=42, max_iter=300)
    kmeans.fit(embeddings_gpu)
    cluster_labels = kmeans.labels_

    # Convert to CPU numpy
    cluster_labels_np = cluster_labels.get()

    # NMI
    nmi = normalized_mutual_info_score(label_ids, cluster_labels_np)

    # Purity (using your ClusteringMetric class)
    cm = ClusteringMetric(label_ids, cluster_labels_np)
    purity = cm.purity_score()

    # Silhouette
    # sil_score = silhouette_score(embeddings_gpu, cluster_labels_np)

    print(f"[{title}] NMI: {nmi:.4f}")
    print(f"[{title}] Purity: {purity:.4f}")
    # print(f"[{title}] Silhouette: {sil_score:.4f}\n")



X_full, X_masked, y = extract_relation_embeddings(
    triples = full_triples,
    tokenizer = tokenizer,
    model = model,
    batch_size=128,
    device="cuda"
)

X_core, y_core = extract_core_relation_embeddings(
    triples=core_rel_triples,
    tokenizer=tokenizer,
    model=model,
    batch_size=128,
    device="cuda"
)

# Step 3: Label encoding
label_encoder = LabelEncoder()

label_ids = label_encoder.fit_transform(y)
core_label_ids = label_encoder.fit_transform(y_core)

print('label_ids', len(label_ids))
print('core_label_ids', len(core_label_ids))
print("")

# Step 5: Run Evaluation
evaluate_embeddings(X_full, y, label_ids, title="Full Relation")
print("")
evaluate_embeddings(X_core, y_core, core_label_ids, title="Core Relation Only")

print("")
evaluate_embeddings(X_masked, y, label_ids, title="Masked Relation")
print("")

Extracting embeddings: 100%|██████████| 48/48 [00:10<00:00,  4.72it/s]


core_rel_triples: 5940


Processing core_rel_triples in batches: 100%|██████████| 47/47 [00:05<00:00,  8.38it/s]


label_ids 6029
core_label_ids 5940

[Full Relation] NMI: 0.8521
[Full Relation] Purity: 0.6374

[Core Relation Only] NMI: 0.8870
[Core Relation Only] Purity: 0.6806

[Masked Relation] NMI: 0.3581
[Masked Relation] Purity: 0.2355



/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:195: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  return func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:195: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  return func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:195: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  return func(*args, **kwargs)


# Existing + Non-existing Triples

In [11]:
all_triples = existing_triples + non_existing_triples
print(len(all_triples))

838350


In [12]:
all_triples_relation_labels = [r for _, r, _ in all_triples]

all_triples_Relations = set(list(all_triples_relation_labels))

all_triples_relations = [r for r in all_triples_Relations if r != 'isa']  # Exclude 'isa'

print(len(all_triples_relations))


# Step 1: Keep only relations with <2 underscores
all_triple_core_rel_relations = [r for r in all_triples_relations if r.count('_') < 2]
all_triples_two_underline_relations = [r for r in all_triples_relations if r.count('_') >= 2]

# # Step 2: Keep only triples with those relations
at_full_triples = [t for t in all_triples if t[1] in all_triples_relations]
at_core_rel_triples = [t for t in all_triples if t[1] in all_triple_core_rel_relations]
# revised_triples = [t for t in existing_triples if t[1] in two_underline_relations]

# print('core_rel_relations', len(core_rel_relations), core_rel_relations)
print('all_triples_two_underline_relations', len(all_triples_two_underline_relations))
print('at_core_rel_triples', len(at_core_rel_triples))
# print(len(revised_triples))
print('at_full_triples', len(at_full_triples))

45
all_triples_two_underline_relations 4
at_core_rel_triples 747225
at_full_triples 820125


In [ ]:
at_X_full, at_X_masked, at_y = extract_relation_embeddings(
    triples = at_full_triples,
    tokenizer = tokenizer,
    model = model,
    batch_size=128,
    device="cuda"
)

at_X_core, at_y_core = extract_core_relation_embeddings(
    triples=at_core_rel_triples,
    tokenizer=tokenizer,
    model=model,
    batch_size=128,
    device="cuda"
)

Extracting embeddings: 100%|██████████| 6408/6408 [23:07<00:00,  4.62it/s]


core_rel_triples: 747225


Processing core_rel_triples in batches: 100%|██████████| 5838/5838 [11:47<00:00,  8.25it/s]


In [ ]:
# Step 3: Label encoding
label_encoder = LabelEncoder()

at_label_ids = label_encoder.fit_transform(at_y)
at_core_label_ids = label_encoder.fit_transform(at_y_core)


# Step 5: Run Evaluation
evaluate_embeddings(at_X_full, at_y, at_label_ids, title="All_triples_Full Relation")
print("")
evaluate_embeddings(at_X_core, at_y_core, at_core_label_ids, title="All_triples_Core Relation Only")

print("")
evaluate_embeddings(at_X_masked, at_y, at_label_ids, title="All_triples_Masked Relation")
print("")

label_ids 6029
core_label_ids 5940



/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:195: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  return func(*args, **kwargs)


[All_triples_Full Relation] NMI: 0.9418
[All_triples_Full Relation] Purity: 0.9281



/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:195: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  return func(*args, **kwargs)


[All_triples_Core Relation Only] NMI: 0.9638
[All_triples_Core Relation Only] Purity: 0.9547

[All_triples_Masked Relation] NMI: 0.0000
[All_triples_Masked Relation] Purity: 0.0451



/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:195: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  return func(*args, **kwargs)


In [ ]:
all_triples_relation_labels = [r for _, r, _ in at_full_triples]
all_triples_core_relation_labels = [r for _, r, _ in at_core_rel_triples]

# all_triples_relation_labels = [r for _, r, _ in all_triples]

all_triples_Relations = set(list(all_triples_relation_labels))

all_triples_relations = [r for r in all_triples_Relations if r != 'isa']  # Exclude 'isa'

# print('label_ids', len(at_label_ids))
# print('core_label_ids', len(at_core_label_ids))
# print("")

In [ ]:
print(len(set(list(all_triples_relation_labels))))
print(len(set(list(all_triples_core_relation_labels))))


45
41


# 0409_MASK

In [20]:
# Load model & tokenizer
fill_mask = pipeline("fill-mask", model="bert-base-uncased", device=0 if torch.cuda.is_available() else -1)


# Use this to encode predicted full sentences
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)
model.to(device)

# === STEP 1: Setup ===
def clean_text(text):
    return text.replace("_", " ")

pair_to_rels = defaultdict(list)
for subj, rel, obj in full_triples:
    pair_to_rels[(clean_text(subj), clean_text(obj))].append(clean_text(rel))

def encode_sentence(text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
# === STEP 2: Predict and encode ===
X_embed = []     # Filled relation embeddings
X_masked = []    # Masked sentence embeddings
y_true = []      # Ground-truth relation
pred_tokens = [] # Predicted relation word

for (subj, obj), rels in tqdm(pair_to_rels.items(), desc="Processing pairs"):
    top_k = len(set(rels))
    mask_prompt = f"{subj} {fill_mask.tokenizer.mask_token} {obj}"

    try:
        predictions = fill_mask(mask_prompt, top_k=top_k)
    except Exception as e:
        print(f"Error on: {mask_prompt}")
        continue

    for pred in predictions:
        token = pred["token_str"].strip()

        filled_sentence = f"{subj} {token} {obj}"
        masked_sentence = f"{subj} {fill_mask.tokenizer.mask_token} {obj}"

        emb_filled = encode_sentence(filled_sentence)
        emb_masked = encode_sentence(masked_sentence)

        X_embed.append(emb_filled)
        X_masked.append(emb_masked)
        y_true.append(clean_text(rels[0]))
        pred_tokens.append(token)

# === STEP 3: Clustering ===
X_embed = np.array(X_embed)
X_masked = np.array(X_masked)

unique_preds = sorted(set(pred_tokens))
unique_gold = sorted(set(y_true))
k1 = len(unique_preds)
k2 = len(unique_gold)

# # Step 5: KMeans clustering
# X_embed = np.array(X_embed)
# n_clusters = len(set(y_true))

# print('n_clstuers', n_clusters)
# print('X_embed', len(X_embed))

# kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# pred_clusters = kmeans.fit_predict(X_embed)

# # Step 6: Evaluate
# nmi = normalized_mutual_info_score(y_true, pred_clusters)
# silhouette = silhouette_score(X_embed, pred_clusters)


# # Purity (using your ClusteringMetric class)
# cm = ClusteringMetric(y_true, pred_clusters)
# purity = cm.purity_score()


# print(f"\nNMI: {nmi:.4f}")
# print(f"\nPurity: {purity:.4f}")
# print(f"Silhouette Score: {silhouette:.4f}")

# Clustering on filled relation embeddings
print("\n🧪 Clustering on FILLED sentence embeddings")
for label, k in [("Predicted", k1), ("Gold", k2)]:
    kmeans = KMeans(n_clusters=k, random_state=42)
    pred_clusters = kmeans.fit_predict(X_embed)

    nmi = normalized_mutual_info_score(y_true, pred_clusters)
    silhouette = silhouette_score(X_embed, pred_clusters)
    purity = ClusteringMetric(y_true, pred_clusters).purity_score()

    print(f"\n📌 Filled Clustering using {label} k={k}:")
    print(f"   🔹 NMI:         {nmi:.4f}")
    print(f"   🔹 Purity:      {purity:.4f}")
    print(f"   🔹 Silhouette:  {silhouette:.4f}")

# Clustering on masked relation prompts
print("\n🧪 Clustering on MASKED sentence embeddings")
for label, k in [("Predicted", k1), ("Gold", k2)]:
    kmeans = KMeans(n_clusters=k, random_state=42)
    pred_clusters = kmeans.fit_predict(X_masked)

    nmi = normalized_mutual_info_score(y_true, pred_clusters)
    silhouette = silhouette_score(X_masked, pred_clusters)
    purity = ClusteringMetric(y_true, pred_clusters).purity_score()

    print(f"\n📌 Masked Clustering using {label} k={k}:")
    print(f"   🔹 NMI:         {nmi:.4f}")
    print(f"   🔹 Purity:      {purity:.4f}")
    print(f"   🔹 Silhouette:  {silhouette:.4f}")

# === STEP 4: Cosine similarity between masked and filled
cos_sim = cosine_similarity(X_embed, X_masked)
mean_diag_sim = np.mean(np.diag(cos_sim))
print(f"\n🔍 Mean Cosine Similarity between filled and masked embeddings: {mean_diag_sim:.4f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Processing pairs: 100%|██████████| 3795/3795 [02:32<00:00, 24.81it/s]



🧪 Clustering on FILLED sentence embeddings

📌 Filled Clustering using Predicted k=240:
   🔹 NMI:         0.3874
   🔹 Purity:      1.0000
   🔹 Silhouette:  0.0992

📌 Filled Clustering using Gold k=44:
   🔹 NMI:         0.2926
   🔹 Purity:      1.0000
   🔹 Silhouette:  0.1041

🧪 Clustering on MASKED sentence embeddings

📌 Masked Clustering using Predicted k=240:
   🔹 NMI:         0.4111
   🔹 Purity:      1.0000
   🔹 Silhouette:  0.1458

📌 Masked Clustering using Gold k=44:
   🔹 NMI:         0.3033
   🔹 Purity:      1.0000
   🔹 Silhouette:  0.1399

🔍 Mean Cosine Similarity between filled and masked embeddings: 0.8763


In [30]:
# Step 4: Clustering Evaluation Function
def evaluate_embeddings(embeddings, labels, label_ids, title=""):
    embeddings_gpu = cp.asarray(embeddings)
    num_clusters = len(set(label_ids))

    kmeans = KMeans(n_clusters=num_clusters, random_state=42, max_iter=300)
    kmeans.fit(embeddings_gpu)
    cluster_labels = kmeans.labels_

    # Convert to CPU numpy
    cluster_labels_np = cluster_labels.get()

    # NMI
    nmi = normalized_mutual_info_score(label_ids, cluster_labels_np)

    # Purity (using your ClusteringMetric class)
    cm = ClusteringMetric(label_ids, cluster_labels_np)
    purity = cm.purity_score()

    # Silhouette
    # sil_score = silhouette_score(embeddings_gpu, cluster_labels_np)

    print(f"[{title}] NMI: {nmi:.4f}")
    print(f"[{title}] Purity: {purity:.4f}")
    # print(f"[{title}] Silhouette: {sil_score:.4f}\n")


# === STEP 3: Clustering ===
X_embed = np.array(X_embed)
X_masked = np.array(X_masked)
# X_embed = cp.asarray(X_embed)
# X_masked = cp.asarray(X_masked)


ll = []
for i, sublist in enumerate(y_true):
    for rel in sublist:
        ll.append(rel)

unique_gold = sorted(set(ll))

unique_preds = sorted(set(pred_tokens))


# Step 3: Label encoding
label_encoder = LabelEncoder()

true_label_ids = label_encoder.fit_transform(y_true)

pred_label_ids = label_encoder.fit_transform(pred_tokens)


print('true_label_ids', len(true_label_ids), len(set(true_label_ids)))
print('pred_label_ids', len(pred_label_ids), len(set(pred_label_ids)))
print("")

# # Step 5: Run Evaluation
evaluate_embeddings(X_embed, y_true, true_label_ids, title="Full Relation")
print("")
# evaluate_embeddings(X_core, y_core, core_label_ids, title="Core Relation Only")

print("")
evaluate_embeddings(X_masked, y_true, true_label_ids, title="using Gold k=45, Masked Relation")
print("")

evaluate_embeddings(X_masked, pred_tokens, pred_label_ids, title="using Predicted k=240, Masked Relation")
print("")

true_label_ids 6029 44
pred_label_ids 6029 240

[Full Relation] NMI: 0.2926
[Full Relation] Purity: 0.2024


[using Gold k=45, Masked Relation] NMI: 0.3033
[using Gold k=45, Masked Relation] Purity: 0.2014

[using Predicted k=240, Masked Relation] NMI: 0.3147
[using Predicted k=240, Masked Relation] Purity: 0.1194



In [21]:
print(k1) # unique_preds
print(k2) # unique_gold
print('pred_tokens', len(pred_tokens))
print('y_true', len(y_true))
print(unique_preds[:20])
print(unique_gold[:20])

240
44
pred_tokens 6029
y_true 6029
['##1', '##a', '##ago', '##al', '##arium', '##ase', '##borne', '##d', '##ea', '##eal', '##ed', '##ia', '##ing', '##ist', '##istic', '##ly', '##s', '##tic', '##uria', '##zia']
['adjacent to', 'affects', 'analyzes', 'assesses effect of', 'associated with', 'carries out', 'causes', 'co-occurs with', 'complicates', 'conceptual part of', 'conceptually related to', 'connected to', 'consists of', 'contains', 'degree of', 'derivative of', 'developmental form of', 'diagnoses', 'disrupts', 'evaluation of']


In [ ]:
print(unique_preds)
print(unique_gold)

['##1', '##a', '##ago', '##al', '##arium', '##ase', '##borne', '##d', '##ea', '##eal', '##ed', '##ia', '##ing', '##ist', '##istic', '##ly', '##s', '##tic', '##uria', '##zia', ',', '-', '.', '/', '1', '2', ':', ';', '=', '?', 'a', 'abnormal', 'abuse', 'acquired', 'activity', 'administration', 'after', 'age', 'an', 'analysis', 'and', 'animal', 'antigen', 'any', 'as', 'attribute', 'b', 'based', 'behavior', 'between', 'biological', 'blood', 'body', 'borne', 'brain', 'by', 'c', 'causing', 'cell', 'chain', 'change', 'chemical', 'classification', 'clinical', 'complex', 'component', 'compound', 'compounds', 'concept', 'condition', 'congenital', 'control', 'culture', 'cycle', 'd', 'damage', 'death', 'deficiency', 'delivery', 'dependence', 'dependency', 'dependent', 'design', 'diagnostic', 'disability', 'discovery', 'disease', 'disorder', 'division', 'drug', 'dysfunction', 'e', 'element', 'emergency', 'enforcement', 'engineering', 'exchange', 'exposure', 'factor', 'family', 'fatal', 'feeding', '

In [ ]:
overlap = set(unique_preds) & set(unique_gold)  # or: set(list1).intersection(list2)

if overlap:
    print("✅ Overlapping items found:", overlap)
else:
    print("❌ No overlap between lists.")

NameError: name 'unique_preds' is not defined

In [17]:

# Setup
fill_mask = pipeline("fill-mask", model="bert-base-uncased", device=0 if torch.cuda.is_available() else -1)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

def clean_text(text):
    return text.replace("_", " ")

# === STEP 1: Prepare triples ===
print("🔍 Step 1: Collecting (subject, object) → [relations] mapping...")
pair_to_rels = defaultdict(list)
for subj, rel, obj in full_triples:
  pair_to_rels[(clean_text(subj), clean_text(obj))].append(clean_text(rel))

print(f"✅ Total unique (subject, object) pairs: {len(pair_to_rels)}")

def encode_sentence(text):
  with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

X_embed = []
y_true = []
pred_tokens = []

max_print_samples = 5
printed_samples = 0

for (subj, obj), rels in tqdm(pair_to_rels.items(), desc="Processing pairs"):
    top_k = len(set(rels))
    mask_prompt = f"{subj} {fill_mask.tokenizer.mask_token} {obj}"

    try:
        predictions = fill_mask(mask_prompt, top_k=top_k)
    except Exception as e:
        print(f"⚠️  Error on: {mask_prompt}")
        continue

    if printed_samples < max_print_samples:
        print(f"\n📎 Input: {mask_prompt}")
        print(f"🔮 Top-{top_k} predicted tokens: {[p['token_str'].strip() for p in predictions]}")
        print(f"🎯 Ground-truth relations: {rels}")

    for pred in predictions:
        token = pred["token_str"].strip()
        sentence = f"{subj} {token} {obj}"
        emb = encode_sentence(sentence)

        if printed_samples < max_print_samples:
            print(f"📘 Encoded Sentence: {sentence}")
            printed_samples += 1

        X_embed.append(emb)
        y_true.append(clean_text(rels[0]))
        pred_tokens.append(token)

# === STEP 3: Clustering with two k values ===
print("\n🧪 Step 3: Clustering and Evaluation")

X_embed = np.array(X_embed)
unique_preds = sorted(set(pred_tokens))
unique_gold = sorted(set(y_true))

k1 = len(unique_preds)  # Based on predicted relation types
k2 = len(unique_gold)   # Based on ground-truth relations

print(f"🔢 Number of unique predicted relations: {k1}")
print(f"🔢 Number of unique ground-truth relations: {k2}")

for label, k in [("Predicted", k1), ("Gold", k2)]:

  print(f"\n🚀 Clustering using {label} k={k}")
  kmeans = KMeans(n_clusters=k, random_state=42)
  pred_clusters = kmeans.fit_predict(X_embed)

  nmi = normalized_mutual_info_score(y_true, pred_clusters)
  silhouette = silhouette_score(X_embed, pred_clusters)

  cm = ClusteringMetric(y_true, pred_clusters)
  purity = cm.purity_score()

  print(f"📊 NMI:         {nmi:.4f}")
  print(f"📊 Purity:      {purity:.4f}")
  print(f"📊 Silhouette:  {silhouette:.4f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


🔍 Step 1: Collecting (subject, object) → [relations] mapping...
✅ Total unique (subject, object) pairs: 3795


Processing pairs:   0%|          | 4/3795 [00:00<01:51, 33.96it/s]


📎 Input: diagnostic procedure [MASK] occupation or discipline
🔮 Top-2 predicted tokens: [':', '-']
🎯 Ground-truth relations: ['method of', 'issue in']
📘 Encoded Sentence: diagnostic procedure : occupation or discipline
📘 Encoded Sentence: diagnostic procedure - occupation or discipline

📎 Input: molecular biology research technique [MASK] mental process
🔮 Top-2 predicted tokens: [':', 'of']
🎯 Ground-truth relations: ['affects', 'measures']
📘 Encoded Sentence: molecular biology research technique : mental process
📘 Encoded Sentence: molecular biology research technique of mental process

📎 Input: mental process [MASK] behavior
🔮 Top-2 predicted tokens: ['and', 'mental']
🎯 Ground-truth relations: ['result of', 'affects']
📘 Encoded Sentence: mental process and behavior


Processing pairs: 100%|██████████| 3795/3795 [01:36<00:00, 39.20it/s]


🧪 Step 3: Clustering and Evaluation
🔢 Number of unique predicted relations: 240
🔢 Number of unique ground-truth relations: 44


ValueError: not enough values to unpack (expected 3, got 2)

In [18]:
print(f"🔢 Number of unique predicted relations: {k1}")
print(f"🔢 Number of unique ground-truth relations: {k2}")

for label, k in [("Predicted", k1), ("Gold", k2)]:

  print(f"\n🚀 Clustering using {label} k={k}")
  kmeans = KMeans(n_clusters=k, random_state=42)
  pred_clusters = kmeans.fit_predict(X_embed)

  nmi = normalized_mutual_info_score(y_true, pred_clusters)
  silhouette = silhouette_score(X_embed, pred_clusters)

  cm = ClusteringMetric(y_true, pred_clusters)
  purity = cm.purity_score()

  print(f"📊 NMI:         {nmi:.4f}")
  print(f"📊 Purity:      {purity:.4f}")
  print(f"📊 Silhouette:  {silhouette:.4f}")


🔢 Number of unique predicted relations: 240
🔢 Number of unique ground-truth relations: 44

🚀 Clustering using Predicted k=240
📊 NMI:         0.3874
📊 Purity:      1.0000
📊 Silhouette:  0.0992

🚀 Clustering using Gold k=44
📊 NMI:         0.2926
📊 Purity:      1.0000
📊 Silhouette:  0.1041


In [ ]:
from transformers import pipeline
from random import sample

random.seed(42)  # ✅ Fix the randomness for reproducibility

# Load fill-mask model for predicting relations
fill_mask = pipeline("fill-mask", model="bert-base-uncased", device=0 if torch.cuda.is_available() else -1)

# Use this to encode predicted full sentences
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)
model.eval()

def clean_text(text):
    return text.replace("_", " ")

def encode_sentence(text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# === Step 1: Count unique relations per (subject, object)
pair_to_rels = defaultdict(list)
for subj, rel, obj in full_triples:
    pair_to_rels[(clean_text(subj), clean_text(obj))].append(rel)

# ✅ Sample 5 (subj, obj) pairs randomly
filtered_pairs = [item for item in pair_to_rels.items() if len(set(item[1])) >= 1]
sampled_pairs = sample(filtered_pairs, k=min(5, len(filtered_pairs)))
sampled_pair_to_rels = dict(sampled_pairs)

# === Step 2: For each (s, o) pair, get top_k predictions and encode them
X_embed = []
y_true = []

print("\n===== Processing Sampled Pairs =====")
for (subj, obj), rels in tqdm(sampled_pair_to_rels.items(), desc="Processing sampled pairs"):
    unique_rels = list(set(rels))
    top_k = len(unique_rels)

    print(f"\n🔹 Processing pair: ({subj} → {obj})")
    print(f"  Ground-truth relations: {unique_rels}")

    # Step 3: Build masked prompt
    mask_prompt = f"{subj} {fill_mask.tokenizer.mask_token} {obj}"
    print(f"  Masked prompt: {mask_prompt}")

    try:
        predictions = fill_mask(mask_prompt, top_k=top_k)
        print(f"  Top-{top_k} predictions:")
        for p in predictions:
            print(f"    {p['token_str']} (score: {p['score']:.4f})")
    except Exception as e:
        print(f"  ❌ Error on: {mask_prompt} — {e}")
        continue

    # Step 4: For each predicted token, construct full sentence and embed
    for pred in predictions:
        pred_token = pred["token_str"]
        pred_sentence = f"{subj} {pred_token} {obj}"
        print(f"    → Encoding sentence: '{pred_sentence}'")

        emb = encode_sentence(pred_sentence)
        print(f"      Embedding norm: {np.linalg.norm(emb):.4f}")
        X_embed.append(emb)
        print('unique_rels', unique_rels)
        # Add the ground-truth relation for this pair (assumes first is true)
        y_true.append(unique_rels[0])

# Step 5: KMeans clustering
print("\n===== Clustering =====")
X_embed = np.array(X_embed)
n_clusters = len(set(y_true))

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
pred_clusters = kmeans.fit_predict(X_embed)

# Step 6: Evaluate
nmi = normalized_mutual_info_score(y_true, pred_clusters)
silhouette = silhouette_score(X_embed, pred_clusters)

print("\n===== Evaluation Results =====")
print(f"Total embeddings: {len(X_embed)}")
print(f"Unique true labels: {len(set(y_true))}")
print(f"NMI: {nmi:.4f}")
print(f"Silhouette Score: {silhouette:.4f}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0



===== Processing Sampled Pairs =====


Processing sampled pairs: 100%|██████████| 5/5 [00:00<00:00, 38.68it/s]


🔹 Processing pair: (lipid → cell function)
  Ground-truth relations: ['affects']
  Masked prompt: lipid [MASK] cell function
  Top-1 predictions:
    t (score: 0.1797)
    → Encoding sentence: 'lipid t cell function'
      Embedding norm: 10.5107
unique_rels ['affects']

🔹 Processing pair: (tissue → organophosphorus compound)
  Ground-truth relations: ['produces']
  Masked prompt: tissue [MASK] organophosphorus compound
  Top-1 predictions:
    ; (score: 0.2462)
    → Encoding sentence: 'tissue ; organophosphorus compound'
      Embedding norm: 9.7856
unique_rels ['produces']

🔹 Processing pair: (pharmacologic substance → congenital abnormality)
  Ground-truth relations: ['treats', 'causes', 'complicates']
  Masked prompt: pharmacologic substance [MASK] congenital abnormality
  Top-3 predictions:
    use (score: 0.1926)
    abuse (score: 0.0973)
    therapy (score: 0.0550)
    → Encoding sentence: 'pharmacologic substance use congenital abnormality'
      Embedding norm: 10.7197
uniqu


/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:195: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  return func(*args, **kwargs)


In [ ]:
# Specify the subject–object pair to check
target_subj = "diagnostic_procedure"
target_obj = "occupation_or_discipline"

# Filter triples for the specific (subject, object) pair
filtered_rels = [rel for s, rel, o in existing_triples if s == target_subj and o == target_obj]

# Count relation frequency
relation_counts = Counter(filtered_rels)

# Display result
for rel, count in relation_counts.items():
    print(f"{rel}: {count}")


# Count how many times the subject–object pair appears
pair_count = sum(1 for s, _, o in existing_triples if s == target_subj and o == target_obj)

print(f"'{target_subj}' → '{target_obj}' appears {pair_count} times")

In [ ]:
from transformers import pipeline

fill_mask = pipeline("fill-mask", model="bert-base-uncased")

result = fill_mask("diagnostic_procedure [MASK]occupation_or_discipline.")
for r in result:
    print(f"{r['token_str']} (score: {r['score']:.4f})")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0


_ (score: 0.8922)
. (score: 0.0512)
, (score: 0.0213)
/ (score: 0.0077)
= (score: 0.0075)


# 0411

In [31]:
from cuml.cluster import KMeans as cuKMeans
from transformers import pipeline

fill_mask = pipeline("fill-mask", model="bert-base-uncased", device=0 if torch.cuda.is_available() else -1)


# === Utility ===
def clean_text(text):
    return text.replace("_", " ").strip()

# === Prepare gold relation groups ===
pair_to_rels = defaultdict(list)
for subj, rel, obj in full_triples:
  pair_to_rels[(clean_text(subj), clean_text(obj))].append(clean_text(rel))

# === Storage ===
X_embed = []
X_masked = []
y_true = []
pred_tokens = []

# === Main Loop ===
for (subj, obj), rels in tqdm(pair_to_rels.items(), desc="Processing pairs"):
  top_k = len(set(rels))
  mask_prompt = f"{subj} [MASK] {obj}"

  try:
    predictions = fill_mask(mask_prompt, top_k=top_k)
  except:
    continue

  # for pred in predictions:
  #   pred_token = pred["token_str"].strip()
  #   pred_tokens.append(pred_token)
  #   y_true.append(rels)  # use first gold relation as label

  for i in range(min(len(predictions), len(rels))):
    pred_token = predictions[i]["token_str"].strip()
    pred_tokens.append(pred_token)
    y_true.append(rels[i])

    # Convert predicted token to same-length [MASK] string
    rel_tokens = tokenizer.tokenize(pred_token)
    masked_rel = " ".join(["[MASK]"])

    # Build sentences
    filled_sentence = f"{subj} {pred_token} {obj}"
    masked_sentence = f"{subj} {masked_rel} {obj}"

    # === Encode both ===
    with torch.no_grad():
      # Filled
      inputs_filled = tokenizer(filled_sentence, return_tensors="pt", padding=True, truncation=True).to(device)
      out_filled = model(**inputs_filled)
      emb_filled = out_filled.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
      X_embed.append(emb_filled)

      # Masked
      inputs_masked = tokenizer(masked_sentence, return_tensors="pt", padding=True, truncation=True).to(device)
      out_masked = model(**inputs_masked)
      token_ids = tokenizer.convert_ids_to_tokens(inputs_masked["input_ids"][0].cpu().tolist())
      mask_indices = [i for i, tok in enumerate(token_ids) if tok == "[MASK]"]

      if mask_indices:
        emb_masked = out_masked.last_hidden_state[0, mask_indices, :].mean(dim=0).cpu().numpy()
        X_masked.append(emb_masked)

# === STEP 3: Clustering ===
X_embed = np.array(X_embed)
X_masked = np.array(X_masked)
# X_embed = cp.asarray(X_embed)
# X_masked = cp.asarray(X_masked)

unique_preds = sorted(set(pred_tokens))

# ll = []
# for i, sublist in enumerate(y_true):
#     for rel in sublist:
#         ll.append(rel)
# print(len(ll))
# print(len(set(ll)))

unique_gold = sorted(set(y_true))
k1 = len(unique_preds)
k2 = len(unique_gold)

# # Clustering on filled relation embeddings
# print("\n🧪 Clustering on FILLED sentence embeddings")
# for label, k in [("Predicted", k1), ("Gold", k2)]:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     # pred_clusters = kmeans.fit_predict(X_embed)
#     # pred_clusters = kmeans.fit_predict(X_embed)

#     nmi = normalized_mutual_info_score(y_true, pred_clusters)
#     silhouette = silhouette_score(X_embed, pred_clusters)
#     purity = ClusteringMetric(y_true, pred_clusters).purity_score()

#     print(f"\n📌 Filled Clustering using {label} k={k}:")
#     print(f"   🔹 NMI:         {nmi:.4f}")
#     print(f"   🔹 Purity:      {purity:.4f}")
#     print(f"   🔹 Silhouette:  {silhouette:.4f}")

# Clustering on masked relation prompts
print("\n🧪 Clustering on MASKED sentence embeddings")
for label, k in [("Predicted", k1), ("Gold", k2)]:
    kmeans = KMeans(n_clusters=k, random_state=42)
    pred_clusters = kmeans.fit_predict(X_masked)

    nmi = normalized_mutual_info_score(y_true, pred_clusters)
    silhouette = silhouette_score(X_masked, pred_clusters)
    purity = ClusteringMetric(y_true, pred_clusters).purity_score()

    print(f"\n📌 Masked Clustering using {label} k={k}:")
    print(f"   🔹 NMI:         {nmi:.4f}")
    print(f"   🔹 Purity:      {purity:.4f}")
    print(f"   🔹 Silhouette:  {silhouette:.4f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Processing pairs: 100%|██████████| 3795/3795 [02:35<00:00, 24.37it/s]



🧪 Clustering on MASKED sentence embeddings

📌 Masked Clustering using Predicted k=240:
   🔹 NMI:         0.4241
   🔹 Purity:      1.0000
   🔹 Silhouette:  0.1599

📌 Masked Clustering using Gold k=45:
   🔹 NMI:         0.3617
   🔹 Purity:      1.0000
   🔹 Silhouette:  0.1244


In [33]:
k1 = len(unique_preds)
k2 = len(unique_gold)
print(k1)
print(k2)

240
45


In [35]:
# Step 4: Clustering Evaluation Function
def evaluate_embeddings(embeddings, labels, label_ids, title=""):
    embeddings_gpu = cp.asarray(embeddings)
    num_clusters = len(set(label_ids))

    kmeans = KMeans(n_clusters=num_clusters, random_state=42, max_iter=300)
    kmeans.fit(embeddings_gpu)
    cluster_labels = kmeans.labels_

    # Convert to CPU numpy
    cluster_labels_np = cluster_labels.get()

    # NMI
    nmi = normalized_mutual_info_score(label_ids, cluster_labels_np)

    # Purity (using your ClusteringMetric class)
    cm = ClusteringMetric(label_ids, cluster_labels_np)
    purity = cm.purity_score()

    # Silhouette
    # sil_score = silhouette_score(embeddings_gpu, cluster_labels_np)

    print(f"[{title}] NMI: {nmi:.4f}")
    print(f"[{title}] Purity: {purity:.4f}")
    # print(f"[{title}] Silhouette: {sil_score:.4f}\n")


# === STEP 3: Clustering ===
X_embed = np.array(X_embed)
X_masked = np.array(X_masked)
# X_embed = cp.asarray(X_embed)
# X_masked = cp.asarray(X_masked)


ll = []
for i, sublist in enumerate(y_true):
    for rel in sublist:
        ll.append(rel)

unique_gold = sorted(set(ll))

unique_preds = sorted(set(pred_tokens))


# unique_gold = sorted(set(y_true))
# k1 = len(unique_preds)
# k2 = len(unique_gold)


# Step 3: Label encoding
label_encoder = LabelEncoder()

true_label_ids = label_encoder.fit_transform(y_true)

pred_label_ids = label_encoder.fit_transform(pred_tokens)


print('true_label_ids', len(true_label_ids))
print('pred_label_ids', len(pred_label_ids))
print("")

# # Step 5: Run Evaluation
# evaluate_embeddings(X_full, y, label_ids, title="Full Relation")
# print("")
# evaluate_embeddings(X_core, y_core, core_label_ids, title="Core Relation Only")

print("")
evaluate_embeddings(X_masked, y_true, true_label_ids, title="using Gold k=45, Masked Relation")
print("")

evaluate_embeddings(X_masked, pred_tokens, pred_label_ids, title="using Predicted k=240, Masked Relation")
print("")

true_label_ids 6029
pred_label_ids 6029


[using Gold k=45, Masked Relation] NMI: 0.3617
[using Gold k=45, Masked Relation] Purity: 0.2518

[using Predicted k=240, Masked Relation] NMI: 0.3884
[using Predicted k=240, Masked Relation] Purity: 0.1582



In [ ]:
from cuml.cluster import KMeans as cuKMeans
from transformers import pipeline

fill_mask = pipeline("fill-mask", model="bert-base-uncased", device=0 if torch.cuda.is_available() else -1)


# === Utility ===
def clean_text(text):
    return text.replace("_", " ").strip()

# === Prepare gold relation groups ===
pair_to_rels = defaultdict(list)
for subj, rel, obj in at_full_triples:
  pair_to_rels[(clean_text(subj), clean_text(obj))].append(clean_text(rel))

# === Storage ===
at_X_embed = []
at_X_masked = []
at_y_true = []
at_pred_tokens = []

# === Main Loop ===
for (subj, obj), rels in tqdm(pair_to_rels.items(), desc="Processing pairs"):
  top_k = len(set(rels))
  mask_prompt = f"{subj} [MASK] {obj}"

  try:
    predictions = fill_mask(mask_prompt, top_k=top_k)
  except:
    continue

  # for pred in predictions:
  #   pred_token = pred["token_str"].strip()
  #   pred_tokens.append(pred_token)
  #   y_true.append(rels)  # use first gold relation as label

  for i in range(min(len(predictions), len(rels))):
    pred_token = predictions[i]["token_str"].strip()
    pred_tokens.append(pred_token)
    at_y_true.append(rels[i])

    # Convert predicted token to same-length [MASK] string
    rel_tokens = tokenizer.tokenize(pred_token)
    masked_rel = " ".join(["[MASK]"])

    # Build sentences
    filled_sentence = f"{subj} {pred_token} {obj}"
    masked_sentence = f"{subj} {masked_rel} {obj}"

    # === Encode both ===
    with torch.no_grad():
      # Filled
      inputs_filled = tokenizer(filled_sentence, return_tensors="pt", padding=True, truncation=True).to(device)
      out_filled = model(**inputs_filled)
      emb_filled = out_filled.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
      X_embed.append(emb_filled)

      # Masked
      inputs_masked = tokenizer(masked_sentence, return_tensors="pt", padding=True, truncation=True).to(device)
      out_masked = model(**inputs_masked)
      token_ids = tokenizer.convert_ids_to_tokens(inputs_masked["input_ids"][0].cpu().tolist())
      mask_indices = [i for i, tok in enumerate(token_ids) if tok == "[MASK]"]

      if mask_indices:
        emb_masked = out_masked.last_hidden_state[0, mask_indices, :].mean(dim=0).cpu().numpy()
        X_masked.append(emb_masked)

# === STEP 3: Clustering ===
at_X_embed = np.array(X_embed)
X_masked = np.array(X_masked)
# X_embed = cp.asarray(X_embed)
# X_masked = cp.asarray(X_masked)

unique_preds = sorted(set(pred_tokens))

# ll = []
# for i, sublist in enumerate(y_true):
#     for rel in sublist:
#         ll.append(rel)
# print(len(ll))
# print(len(set(ll)))

unique_gold = sorted(set(y_true))
k1 = len(unique_preds)
k2 = len(unique_gold)

# # Clustering on filled relation embeddings
# print("\n🧪 Clustering on FILLED sentence embeddings")
# for label, k in [("Predicted", k1), ("Gold", k2)]:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     # pred_clusters = kmeans.fit_predict(X_embed)
#     # pred_clusters = kmeans.fit_predict(X_embed)

#     nmi = normalized_mutual_info_score(y_true, pred_clusters)
#     silhouette = silhouette_score(X_embed, pred_clusters)
#     purity = ClusteringMetric(y_true, pred_clusters).purity_score()

#     print(f"\n📌 Filled Clustering using {label} k={k}:")
#     print(f"   🔹 NMI:         {nmi:.4f}")
#     print(f"   🔹 Purity:      {purity:.4f}")
#     print(f"   🔹 Silhouette:  {silhouette:.4f}")

# Clustering on masked relation prompts
# print("\n🧪 Clustering on MASKED sentence embeddings")
# for label, k in [("Predicted", k1), ("Gold", k2)]:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     pred_clusters = kmeans.fit_predict(X_masked)

#     nmi = normalized_mutual_info_score(y_true, pred_clusters)
#     silhouette = silhouette_score(X_masked, pred_clusters)
#     purity = ClusteringMetric(y_true, pred_clusters).purity_score()

#     print(f"\n📌 Masked Clustering using {label} k={k}:")
#     print(f"   🔹 NMI:         {nmi:.4f}")
#     print(f"   🔹 Purity:      {purity:.4f}")
#     print(f"   🔹 Silhouette:  {silhouette:.4f}")


In [ ]:
flat_list = [item for sublist in y_true for item in sublist]
counts = Counter(flat_list)

print("Total relations:", len(flat_list))  # 👈 total number of elements
print("Unique relations:", len(counts))    # 👈 how many different ones

# # Optional: show frequency of each
# for relation, count in counts.items():
#     print(f"{relation}: {count}")
# ll = []
# for i, sublist in enumerate(y_true):
#     for rel in sublist:
#         ll.append(rel)
# print(len(ll))
# print(len(set(ll)))

Total relations: 57761
Unique relations: 26


In [ ]:
print(len(X_masked))
print(len(full_triples))
print('masked_sentence', len(masked_sentence))
print('unique_preds_k1', k1)
print('unique_y_true_k2', k2) # "interconnects" is gone
print('pred_tokens', len(pred_tokens))
print('y_true', len(y_true))
print(len(set(y_true)))

6029
6029
masked_sentence 41
unique_preds_k1 240
unique_y_true_k2 45
pred_tokens 6029
y_true 6029
45


In [19]:
# Step 4: Clustering Evaluation Function
def evaluate_embeddings(embeddings, labels, label_ids, title=""):
    embeddings_gpu = cp.asarray(embeddings)
    num_clusters = len(set(label_ids))

    kmeans = KMeans(n_clusters=num_clusters, random_state=42, max_iter=300)
    kmeans.fit(embeddings_gpu)
    cluster_labels = kmeans.labels_

    # Convert to CPU numpy
    cluster_labels_np = cluster_labels.get()

    # NMI
    nmi = normalized_mutual_info_score(label_ids, cluster_labels_np)

    # Purity (using your ClusteringMetric class)
    cm = ClusteringMetric(label_ids, cluster_labels_np)
    purity = cm.purity_score()

    # Silhouette
    # sil_score = silhouette_score(embeddings_gpu, cluster_labels_np)

    print(f"[{title}] NMI: {nmi:.4f}")
    print(f"[{title}] Purity: {purity:.4f}")
    # print(f"[{title}] Silhouette: {sil_score:.4f}\n")


# === STEP 3: Clustering ===
X_embed = np.array(X_embed)
X_masked = np.array(X_masked)
# X_embed = cp.asarray(X_embed)
# X_masked = cp.asarray(X_masked)


ll = []
for i, sublist in enumerate(y_true):
    for rel in sublist:
        ll.append(rel)

unique_gold = sorted(set(ll))

unique_preds = sorted(set(pred_tokens))


# unique_gold = sorted(set(y_true))
# k1 = len(unique_preds)
# k2 = len(unique_gold)


# Step 3: Label encoding
label_encoder = LabelEncoder()

true_label_ids = label_encoder.fit_transform(y_true)

pred_label_ids = label_encoder.fit_transform(pred_tokens)


print('true_label_ids', len(true_label_ids))
print('pred_label_ids', len(pred_label_ids))
print("")

# # Step 5: Run Evaluation
# evaluate_embeddings(X_full, y, label_ids, title="Full Relation")
# print("")
# evaluate_embeddings(X_core, y_core, core_label_ids, title="Core Relation Only")

print("")
evaluate_embeddings(X_masked, y_true, true_label_ids, title="using Gold k=45, Masked Relation")
print("")

# evaluate_embeddings(X_masked, pred_tokens, pred_label_ids, title="using Predicted k=240, Masked Relation")
# print("")

true_label_ids 6029
pred_label_ids 6029




ValueError: Found input variables with inconsistent numbers of samples: [6029, 920]

# 0413

In [36]:
from cuml.cluster import KMeans as cuKMeans
from transformers import pipeline

fill_mask = pipeline("fill-mask", model="bert-base-uncased", device=0 if torch.cuda.is_available() else -1)


# === Utility ===
def clean_text(text):
    return text.replace("_", " ").strip()

batch_size = 32


# === Prepare gold relation groups ===
pair_to_rels = defaultdict(list)
for subj, rel, obj in at_full_triples:
  pair_to_rels[(clean_text(subj), clean_text(obj))].append(clean_text(rel))

# === Storage ===
at_X_embed = []
at_X_masked = []
at_y_true = []
at_pred_tokens = []

# === Main Loop ===
for (subj, obj), rels in tqdm(pair_to_rels.items(), desc="Processing pairs"):
    top_k = len(set(rels))
    mask_prompt = f"{subj} [MASK] {obj}"

    try:
        predictions = fill_mask(mask_prompt, top_k=top_k)
    except:
        continue

    # Accumulate a batch
    filled_batch = []
    masked_batch = []
    batch_y = []
    batch_preds = []

    for i in range(min(len(predictions), len(rels))):
        pred_token = predictions[i]["token_str"].strip()
        filled_sentence = f"{subj} {pred_token} {obj}"
        masked_sentence = f"{subj} [MASK] {obj}"

        filled_batch.append(filled_sentence)
        masked_batch.append(masked_sentence)
        batch_y.append(rels[i])
        batch_preds.append(pred_token)

        # When batch is full or it's the last one
        if len(filled_batch) == batch_size or i == min(len(predictions), len(rels)) - 1:
            # === Encode both batches ===
            with torch.no_grad():
                inputs_filled = tokenizer(filled_batch, return_tensors="pt", padding=True, truncation=True).to(device)
                outputs_filled = model(**inputs_filled)

                inputs_masked = tokenizer(masked_batch, return_tensors="pt", padding=True, truncation=True).to(device)
                outputs_masked = model(**inputs_masked)

            for j in range(len(filled_batch)):
                emb_filled = outputs_filled.last_hidden_state[j].mean(dim=0).cpu().numpy()
                emb_masked = outputs_masked.last_hidden_state[j]
                mask_indices = (inputs_masked["input_ids"][j] == tokenizer.mask_token_id).nonzero(as_tuple=True)[0]

                if len(mask_indices) > 0:
                    emb_masked_mean = emb_masked[mask_indices].mean(dim=0).cpu().numpy()
                    at_X_masked.append(emb_masked_mean)

                at_X_embed.append(emb_filled)
                at_y_true.append(batch_y[j])
                at_pred_tokens.append(batch_preds[j])

            # Reset batch
            filled_batch = []
            masked_batch = []
            batch_y = []
            batch_preds = []


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Processing pairs: 100%|██████████| 18225/18225 [24:20<00:00, 12.48it/s]


In [46]:
label_encoder = LabelEncoder()

at_true_label_ids = label_encoder.fit_transform(at_y_true)

at_pred_label_ids = label_encoder.fit_transform(at_pred_tokens)

print('true_label_ids', len(at_true_label_ids))
print('at_pred_label_ids', len(at_pred_label_ids))
print('pred_label_ids', len(pred_label_ids))
print('len', len(set(pred_label_ids)))
print('leneln', len(set(at_pred_label_ids)))
print(len(at_X_masked))
print(len(set(at_true_label_ids)))

true_label_ids 820125
at_pred_label_ids 820125
pred_label_ids 6029
len 240
leneln 6261
820125
45


In [48]:
# Step 4: Clustering Evaluation Function
def evaluate_embeddings(embeddings, label_ids, title=""):
    embeddings_gpu = cp.asarray(embeddings)
    num_clusters = len(set(label_ids))

    kmeans = KMeans(n_clusters=num_clusters, random_state=42, max_iter=300)
    kmeans.fit(embeddings_gpu)
    cluster_labels = kmeans.labels_

    # Convert to CPU numpy
    cluster_labels_np = cluster_labels.get()

    # NMI
    nmi = normalized_mutual_info_score(label_ids, cluster_labels_np)

    # Purity (using your ClusteringMetric class)
    cm = ClusteringMetric(label_ids, cluster_labels_np)
    purity = cm.purity_score()

    # Silhouette
    sil_score = silhouette_score(embeddings_gpu, cluster_labels_np)

    print(f"[{title}] NMI: {nmi:.4f}")
    print(f"[{title}] Purity: {purity:.4f}")

    print(f"[{title}] Silhouette: {sil_score:.4f}\n")



# Step 4: Clustering Evaluation Function
def evaluate_embeddings2(embeddings, label_ids, title=""):
    embeddings_gpu = cp.asarray(embeddings)
    num_clusters = 240

    kmeans = KMeans(n_clusters=num_clusters, random_state=42, max_iter=300)
    kmeans.fit(embeddings_gpu)
    cluster_labels = kmeans.labels_

    # Convert to CPU numpy
    cluster_labels_np = cluster_labels.get()

    # NMI
    nmi = normalized_mutual_info_score(label_ids, cluster_labels_np)

    # Purity (using your ClusteringMetric class)
    cm = ClusteringMetric(label_ids, cluster_labels_np)
    purity = cm.purity_score()

    # Silhouette
    sil_score = silhouette_score(embeddings_gpu, cluster_labels_np)

    print(f"[{title}] NMI: {nmi:.4f}")
    print(f"[{title}] Purity: {purity:.4f}")

    print(f"[{title}] Silhouette: {sil_score:.4f}\n")

In [49]:
label_encoder = LabelEncoder()

at_true_label_ids = label_encoder.fit_transform(at_y_true) # 45

at_pred_label_ids = label_encoder.fit_transform(at_pred_tokens) # 6261 => 240

print('at_true_label_ids', len(set(at_true_label_ids)))
print('at_pred_label_ids', len(set(at_pred_label_ids)))


# # Step 5: Run Evaluation
# evaluate_embeddings(X_full, y, label_ids, title="Full Relation")
# print("")
# evaluate_embeddings(X_core, y_core, core_label_ids, title="Core Relation Only")

# evaluate_embeddings(at_X_masked, at_pred_tokens, at_pred_label_ids, title="using Predicted k=6261, Masked Relation")
# print("")

evaluate_embeddings2(at_X_masked, at_pred_label_ids, title="using Predicted k=240 from original knowledge graph's BERT prediction, Masked Relation")
print("")


evaluate_embeddings(at_X_masked, at_true_label_ids, title="using Gold k=45, Masked Relation")
print("")

at_true_label_ids 45
at_pred_label_ids 6261
[using Predicted k=240 from original knowledge graph's BERT prediction, Masked Relation] NMI: 0.2119
[using Predicted k=240 from original knowledge graph's BERT prediction, Masked Relation] Purity: 0.1123
[using Predicted k=240 from original knowledge graph's BERT prediction, Masked Relation] Silhouette: 0.1046


[using Gold k=45, Masked Relation] NMI: 0.0000
[using Gold k=45, Masked Relation] Purity: 0.0568
[using Gold k=45, Masked Relation] Silhouette: 0.0629




In [ ]:
label_encoder = LabelEncoder()

at_pred_label_ids = label_encoder.fit_transform(at_pred_tokens)
print('at_pred_tokens', len(at_pred_tokens))
print('at_pred_label_ids', len(at_pred_label_ids))

at_pred_tokens 820125
at_pred_label_ids 820125


In [ ]:
evaluate_embeddings(at_X_masked, at_pred_tokens, at_pred_label_ids, title="using Predicted k=6261, Masked Relation")


MemoryError: std::bad_alloc: CUDA error at: /tmp/pip-build-env-c7imy8iv/normal/lib/python3.11/site-packages/librmm/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorIllegalAddress an illegal memory access was encountered

In [ ]:
print('at_true_label_ids', len(at_true_label_ids))
print('at_pred_label_ids', len(at_pred_label_ids))
print(len(set(at_pred_label_ids)))
print(len(set(at_true_label_ids)))

NameError: name 'at_true_label_ids' is not defined

In [ ]:
print(len(set(pred_tokens)))
print(len(pred_tokens))
'''
true_label_ids 6029
pred_label_ids 6029

using Gold k=44, Masked Relation
[Masked Relation] NMI: 0.3787
[Masked Relation] Purity: 0.2795

using Predicted k=240, Masked Relation
[Masked Relation] NMI: 0.3884
[Masked Relation] Purity: 0.1582
'''

240
6029


In [ ]:
from cuml.cluster import KMeans as cuKMeans

# === Utility ===
def clean_text(text):
    return text.replace("_", " ").strip()

# === Prepare gold relation groups ===
pair_to_rels = defaultdict(list)
for subj, rel, obj in full_triples[:21]:
    pair_to_rels[(clean_text(subj), clean_text(obj))].append(clean_text(rel))

# === Storage ===
X_embed = []
X_masked = []
y_true = []
pred_tokens = []

# === Main Loop ===
for (subj, obj), rels in tqdm(pair_to_rels.items(), desc="Processing pairs"):
    top_k = len(set(rels))
    mask_prompt = f"{subj} [MASK] {obj}"

    try:
        predictions = fill_mask(mask_prompt, top_k=top_k)
    except:
        continue

    for pred in predictions:
        pred_token = pred["token_str"].strip()
        pred_tokens.append(pred_token)
        y_true.append(rels[0])  # use first gold relation as label

        # Convert predicted token to same-length [MASK] string
        rel_tokens = tokenizer.tokenize(pred_token)
        masked_rel = " ".join(["[MASK]"])

        # Build sentences
        filled_sentence = f"{subj} {pred_token} {obj}"
        masked_sentence = f"{subj} {masked_rel} {obj}"
        print('\n masked_sentence:', masked_sentence)
        # === Encode both ===
        with torch.no_grad():

          # Masked
          inputs_masked = tokenizer(masked_sentence, return_tensors="pt", padding=True, truncation=True).to(device)
          print('\n inputs_masked', inputs_masked)
          out_masked = model(**inputs_masked)
          token_ids = tokenizer.convert_ids_to_tokens(inputs_masked["input_ids"][0].cpu().tolist())
          print('\n token_ids', token_ids)
          mask_indices = [i for i, tok in enumerate(token_ids) if tok == "[MASK]"]
          print('\nmask_indices', mask_indices)
          if mask_indices:
              emb_masked = out_masked.last_hidden_state[0, mask_indices, :].mean(dim=0).cpu().numpy()
              X_masked.append(emb_masked)

# === STEP 3: Clustering ===
X_embed = np.array(X_embed)
X_masked = np.array(X_masked)

unique_preds = sorted(set(pred_tokens))
unique_gold = sorted(set(y_true))
k1 = len(unique_preds)
k2 = len(unique_gold)

# Clustering on filled relation embeddings
print("\n🧪 Clustering on FILLED sentence embeddings")
for label, k in [("Predicted", k1), ("Gold", k2)]:
    kmeans = KMeans(n_clusters=k, random_state=42)
    pred_clusters = kmeans.fit_predict(X_embed)

    nmi = normalized_mutual_info_score(y_true, pred_clusters)
    silhouette = silhouette_score(X_embed, pred_clusters)
    purity = ClusteringMetric(y_true, pred_clusters).purity_score()

    print(f"\n📌 Filled Clustering using {label} k={k}:")
    print(f"   🔹 NMI:         {nmi:.4f}")
    print(f"   🔹 Purity:      {purity:.4f}")
    print(f"   🔹 Silhouette:  {silhouette:.4f}")

# Clustering on masked relation prompts
print("\n🧪 Clustering on MASKED sentence embeddings")
for label, k in [("Predicted", k1), ("Gold", k2)]:
    kmeans = KMeans(n_clusters=k, random_state=42)
    pred_clusters = kmeans.fit_predict(X_masked)

    nmi = normalized_mutual_info_score(y_true, pred_clusters)
    silhouette = silhouette_score(X_masked, pred_clusters)
    purity = ClusteringMetric(y_true, pred_clusters).purity_score()

    print(f"\n📌 Masked Clustering using {label} k={k}:")
    print(f"   🔹 NMI:         {nmi:.4f}")
    print(f"   🔹 Purity:      {purity:.4f}")
    print(f"   🔹 Silhouette:  {silhouette:.4f}")


Processing pairs:  24%|██▍       | 5/21 [00:00<00:00, 44.62it/s]


 masked_sentence: diagnostic procedure [MASK] occupation or discipline

 inputs_masked {'input_ids': tensor([[  101, 16474,  7709,   103,  6139,  2030,  9009,   102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

 token_ids ['[CLS]', 'diagnostic', 'procedure', '[MASK]', 'occupation', 'or', 'discipline', '[SEP]']

mask_indices [3]

 masked_sentence: molecular biology research technique [MASK] mental process

 inputs_masked {'input_ids': tensor([[ 101, 8382, 7366, 2470, 6028,  103, 5177, 2832,  102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

 token_ids ['[CLS]', 'molecular', 'biology', 'research', 'technique', '[MASK]', 'mental', 'process', '[SEP]']

mask_indices [5]

 masked_sentence: mental process [MASK] behavior

 inputs_masked {'i

Processing pairs:  71%|███████▏  | 15/21 [00:00<00:00, 46.22it/s]


 token_ids ['[CLS]', 'therapeutic', 'or', 'prevent', '##ive', 'procedure', '[MASK]', 'cell', 'function', '[SEP]']

mask_indices [6]

 masked_sentence: professional or occupational group [MASK] diagnostic procedure

 inputs_masked {'input_ids': tensor([[  101,  2658,  2030, 16928,  2177,   103, 16474,  7709,   102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

 token_ids ['[CLS]', 'professional', 'or', 'occupational', 'group', '[MASK]', 'diagnostic', 'procedure', '[SEP]']

mask_indices [5]

 masked_sentence: enzyme [MASK] organ or tissue function

 inputs_masked {'input_ids': tensor([[ 101, 9007,  103, 5812, 2030, 8153, 3853,  102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

 token_ids ['[CLS]', 'enzyme', '[MASK]', 'organ', 'or', 'tissue', '

Processing pairs: 100%|██████████| 21/21 [00:00<00:00, 45.84it/s]
